In [1]:
library(tidyverse)
library(lubridate)
library(fastDummies)
library(tidymodels)
library(distributions3)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.1     ✔ rsample      1.1.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.3     ✔ workflows    1.0.0
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.1     ✔ yardstick    1.1.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()  

ERROR: Error in library(distributions3): there is no package called ‘distributions3’


In [51]:
data = read.csv('https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/data/world_cup%26friendlies_football_en.csv', header= T, sep= ';')
#fifa_ranking = read.csv("https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/FIFA_Ranking.csv", header = TRUE, sep = ",", stringsAsFactors=FALSE, fileEncoding="latin1")

In [54]:
# train test

data %>%
    filter(game_name != 'World Cup 2022' 
           & date_of_play < ('2022-11-20')
          ) -> train

data %>%
    filter(game_name == 'World Cup 2022') -> test

In [55]:
train

country,fifa_date,Matchday,Time,Venue,Opponent,System_of_play,Attendance,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,rank_opponent,Rank,Venue_A,Venue_H,Venue_N
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>
Argentina,2009-12-16,NA,2:00 AM,H,Costa Rica,4-4-2 Diamond,19.000,3,2,International Friendlies,3,2,2010-01-27,44,8,0,1,0
Argentina,2010-02-03,NA,2:00 AM,H,Jamaica,4-4-2 Diamond,20.000,2,1,International Friendlies,2,1,2010-02-11,78,8,0,1,0
Argentina,2010-03-03,NA,8:45 PM,A,Germany,4-4-2 double 6,65.152,0,1,International Friendlies,1,0,2010-03-03,5,9,1,0,0
Argentina,2010-04-28,NA,11:00 PM,H,Haiti,4-4-2 Diamond,,4,0,International Friendlies,4,0,2010-05-05,87,7,0,1,0
Argentina,2010-04-28,NA,9:00 PM,H,Canada,4-4-2 Diamond,52.000,5,0,International Friendlies,5,0,2010-05-24,63,7,0,1,0
Argentina,2010-05-26,Group B,4:00 PM,N,Nigeria,4-4-2 Diamond,55.686,1,0,World Cup 2010,1,0,2010-06-12,21,7,0,0,1
Argentina,2010-05-26,Group B,1:30 PM,N,South Korea,4-4-2 Diamond,82.174,4,1,World Cup 2010,4,1,2010-06-17,47,7,0,0,1
Argentina,2010-05-26,Group B,8:30 PM,N,Greece,4-4-2 Diamond,38.891,0,2,World Cup 2010,2,0,2010-06-22,13,7,0,0,1
Argentina,2010-05-26,Round of 16,8:30 PM,N,Mexico,4-4-2 Diamond,84.377,3,1,World Cup 2010,3,1,2010-06-27,17,7,0,0,1


In [58]:
#train
data

model = 
    glm(
        goals_country ~ Rank + rank_opponent + Venue_H + Venue_N,
        data,
        family = poisson(link = 'log')
       )
summary(model)

country,fifa_date,Matchday,Time,Venue,Opponent,System_of_play,Attendance,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,rank_opponent,Rank,Venue_A,Venue_H,Venue_N
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>
Argentina,2009-12-16,NA,2:00 AM,H,Costa Rica,4-4-2 Diamond,19.000,3,2,International Friendlies,3,2,2010-01-27,44,8,0,1,0
Argentina,2010-02-03,NA,2:00 AM,H,Jamaica,4-4-2 Diamond,20.000,2,1,International Friendlies,2,1,2010-02-11,78,8,0,1,0
Argentina,2010-03-03,NA,8:45 PM,A,Germany,4-4-2 double 6,65.152,0,1,International Friendlies,1,0,2010-03-03,5,9,1,0,0
Argentina,2010-04-28,NA,11:00 PM,H,Haiti,4-4-2 Diamond,,4,0,International Friendlies,4,0,2010-05-05,87,7,0,1,0
Argentina,2010-04-28,NA,9:00 PM,H,Canada,4-4-2 Diamond,52.000,5,0,International Friendlies,5,0,2010-05-24,63,7,0,1,0
Argentina,2010-05-26,Group B,4:00 PM,N,Nigeria,4-4-2 Diamond,55.686,1,0,World Cup 2010,1,0,2010-06-12,21,7,0,0,1
Argentina,2010-05-26,Group B,1:30 PM,N,South Korea,4-4-2 Diamond,82.174,4,1,World Cup 2010,4,1,2010-06-17,47,7,0,0,1
Argentina,2010-05-26,Group B,8:30 PM,N,Greece,4-4-2 Diamond,38.891,0,2,World Cup 2010,2,0,2010-06-22,13,7,0,0,1
Argentina,2010-05-26,Round of 16,8:30 PM,N,Mexico,4-4-2 Diamond,84.377,3,1,World Cup 2010,3,1,2010-06-27,17,7,0,0,1



Call:
glm(formula = goals_country ~ Rank + rank_opponent + Venue_H + 
    Venue_N, family = poisson(link = "log"), data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.8266  -1.1876  -0.1969   0.5617   3.7318  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    0.1958458  0.0399740   4.899 9.62e-07 ***
Rank          -0.0088665  0.0007454 -11.895  < 2e-16 ***
rank_opponent  0.0071007  0.0004399  16.141  < 2e-16 ***
Venue_H        0.2695579  0.0377726   7.136 9.58e-13 ***
Venue_N        0.1828919  0.0612309   2.987  0.00282 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 3201.9  on 2222  degrees of freedom
Residual deviance: 2807.4  on 2218  degrees of freedom
  (176 observations deleted due to missingness)
AIC: 7001.7

Number of Fisher Scoring iterations: 5
